In [28]:
import pandas as pd
import os
import xlrd

In [30]:
path = 'combined_excel (3).xlsx'
df = pd.read_excel(path,index_col=0, engine='openpyxl')

In [31]:
df[['50']]

,50
3,OJUS PETROCHEMICALS LLPC 289 NIRALA NAGAR LUCK...
4,AHMED SAEED AFIFI FACTORY CO. FORRESINS LTD. P...
5,MINH KHANG CHEMICAL TRADINGJOINT STOCK COMPANY...
7,"CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-..."
9,"CHORI CO., LTD.TEL NO.81-76-232-3008FAX NO.81-..."
...,...
8403,"HENGLI PETROCHEMICAL CO., LIMITED_x000D_\nFLAT..."
8413,"MARUBENI INTEX CO.,LTD._x000D_\n1-2-1 DOJIMAHA..."
8414,"XIAMEN GOLDEN CHEMICALS CO., LTD_x000D_\nADDRE..."
8415,"XIAMEN GOLDEN CHEMICALS CO., LTD_x000D_\nADDRE..."


# 第一種case

In [36]:
case1 = {}
for k,v in zip(df['50'].index,df['50'].values):
    if "_x000D" in v:
        case1[k] = v.split('_x000D')[0][:-1]
res1 = pd.DataFrame()
res1['string_X'] = df.loc[case1.keys(),'50'].values
res1['Y_label'] = case1.values()
res1

,string_X,Y_label
0,"KINGFA SCI. AND TECH. CO., LTD._x000D_\nADDRES...","KINGFA SCI. AND TECH. CO., LTD"
1,"LANDMARK INT'L CORP._x000D_\nSUITE 310,GRIFFIT...",LANDMARK INT'L CORP
2,DVA PLASTICS GmbH_x000D_\nVALENTINSKAMP 70_x00...,DVA PLASTICS Gmb
3,SHANGHAI KINGFA SCI. AND TECH._x000D_\nDVPT. C...,SHANGHAI KINGFA SCI. AND TECH
4,SHANGHAI KINGFA SCI. AND TECH._x000D_\nDVPT. C...,SHANGHAI KINGFA SCI. AND TECH
...,...,...
3349,"HENGLI PETROCHEMICAL CO., LIMITED_x000D_\nFLAT...","HENGLI PETROCHEMICAL CO., LIMITE"
3350,"MARUBENI INTEX CO.,LTD._x000D_\n1-2-1 DOJIMAHA...","MARUBENI INTEX CO.,LTD"
3351,"XIAMEN GOLDEN CHEMICALS CO., LTD_x000D_\nADDRE...","XIAMEN GOLDEN CHEMICALS CO., LT"
3352,"XIAMEN GOLDEN CHEMICALS CO., LTD_x000D_\nADDRE...","XIAMEN GOLDEN CHEMICALS CO., LT"


# 去除不必要的東西

In [37]:
df = res1

In [38]:
import re

def preprocess(x):
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 1.去除中文
    x = re.sub('[’!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~，。,.]', '', x) # 2.去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 3.去除換行符號
    x = str.strip(x) # 4.移除左右空白
    return x

df['string_X'] = df['string_X'].apply(preprocess)
df['Y_label'] = df['Y_label'].apply(preprocess)
df.head()

,string_X,Y_label
0,KINGFA SCI AND TECH CO LTDx000DADDRESS NO33 KE...,KINGFA SCI AND TECH CO LTD
1,LANDMARK INTL CORPx000DSUITE 310GRIFFITH CORPO...,LANDMARK INTL CORP
2,DVA PLASTICS GmbHx000DVALENTINSKAMP 70x000DHAM...,DVA PLASTICS Gmb
3,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH
4,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH


# 清洗

In [39]:
df = df.dropna(axis=0).reset_index(drop=True)
print(df.shape)
keep_lst = []
for i in range(len(df)):
    if df.iloc[i]['Y_label'] in df.iloc[i]['string_X']:
        keep_lst.append(i)
df = df.iloc[keep_lst]
print(df.shape)

(3354, 2)
(3354, 2)


# 起始結束標註

In [41]:
def str2index(context,string):
    ys = context.find(string)
    ye = ys + len(string)
    return ys,ye

ys_lst = []
ye_lst = []
for i in range(len(df)):
    context= df['string_X'].values[i]
    string = df['Y_label'].values[i]
    ys,ye = str2index(context,string)
    ys_lst.append(ys)
    ye_lst.append(ye)
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
df.head()

,string_X,Y_label,string_Y_1,string_Y_2
0,KINGFA SCI AND TECH CO LTDx000DADDRESS NO33 KE...,KINGFA SCI AND TECH CO LTD,0,26
1,LANDMARK INTL CORPx000DSUITE 310GRIFFITH CORPO...,LANDMARK INTL CORP,0,18
2,DVA PLASTICS GmbHx000DVALENTINSKAMP 70x000DHAM...,DVA PLASTICS Gmb,0,16
3,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28
4,SHANGHAI KINGFA SCI AND TECHx000DDVPT COLTDx00...,SHANGHAI KINGFA SCI AND TECH,0,28


In [42]:
print(df.shape)

(3354, 4)


In [43]:
df.to_csv('preprocess_for_SQUAD_開狀人.csv')